In [ ]:
!pip install imutils 
!pip install gdown

In [2]:
!git clone https://github.com/shilu10/MIRNet-TF2.git

Cloning into 'MIRNet-TF2'...
remote: Enumerating objects: 445, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 445 (delta 108), reused 100 (delta 45), pack-reused 282
Receiving objects: 100% (445/445), 55.86 MiB | 18.42 MiB/s, done.
Resolving deltas: 100% (248/248), done.


In [3]:
!mv MIRNet-TF2 MIRNet_TF0

In [ ]:
!mv MIRNet_TF8/SIDD_Small_sRGB_Only MIRNet_TF9

In [8]:
!python train_enhancement.py --loss_function=charbonnier --n_epochs=200 --batch_size=10 --num_rrg=2 --num_mrb=2

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [ ]:
import cv2             

In [ ]:
cv2.imread('GT_SRGB_010.PNG').shape

In [ ]:
!pip install imutils 
!pip install gdown

In [ ]:
import gdown 
import os 
import shutil 
from imutils import paths 
import glob 
import glob 
import numpy as np 
from tensorflow import keras 
import tensorflow as tf 
from tensorflow.keras import *


from tensorflow import keras 
import tensorflow as tf 
import numpy as np 


def random_crop(lr_img, hr_img, hr_crop_size=128):
    lr_crop_size = hr_crop_size
    lr_img_shape = tf.shape(lr_img)[:2]

    lr_w = tf.random.uniform(shape=(), maxval=lr_img_shape[1] - lr_crop_size + 1, dtype=tf.int32)
    lr_h = tf.random.uniform(shape=(), maxval=lr_img_shape[0] - lr_crop_size + 1, dtype=tf.int32)

    hr_w = lr_w
    hr_h = lr_h

    lr_img_cropped = lr_img[lr_h:lr_h + lr_crop_size, lr_w:lr_w + lr_crop_size]
    hr_img_cropped = hr_img[hr_h:hr_h + hr_crop_size, hr_w:hr_w + hr_crop_size]

    return lr_img_cropped, hr_img_cropped


def random_flip(lr_img, hr_img):
    rn = tf.random.uniform(shape=(), maxval=1)
    return tf.cond(rn < 0.5,
                   lambda: (lr_img, hr_img),
                   lambda: (tf.image.flip_left_right(lr_img),
                            tf.image.flip_left_right(hr_img)))


def random_rotate(lr_img, hr_img):
    rn = tf.random.uniform(shape=(), maxval=4, dtype=tf.int32)
    return tf.image.rot90(lr_img, rn), tf.image.rot90(hr_img, rn)


def random_crop_sr(lr_img, hr_img, hr_crop_size=96, scale=2):
    lr_crop_size = hr_crop_size // scale
    lr_img_shape = tf.shape(lr_img)[:2]

    lr_w = tf.random.uniform(shape=(), maxval=lr_img_shape[1] - lr_crop_size + 1, dtype=tf.int32)
    lr_h = tf.random.uniform(shape=(), maxval=lr_img_shape[0] - lr_crop_size + 1, dtype=tf.int32)

    hr_w = lr_w * scale
    hr_h = lr_h * scale

    lr_img_cropped = lr_img[lr_h:lr_h + lr_crop_size, lr_w:lr_w + lr_crop_size]
    hr_img_cropped = hr_img[hr_h:hr_h + hr_crop_size, hr_w:hr_w + hr_crop_size]

    return lr_img_cropped, hr_img_cropped


class UnsuuportedFileExtension(Exception):
    def __init__(self, message):
        self.message = message

        
class InitializationErro(Exception):
    def __init__(self, message):
        self.message = message
        

import gdown 
import os 
import shutil 
from imutils import paths 
import glob 
import glob 
import numpy as np 
from tensorflow import keras 
import tensorflow as tf 
from tensorflow.keras import *



class UnsuuportedFileExtension(Exception):
    def __init__(self, message):
        self.message = message

        
class InitializationErro(Exception):
    def __init__(self, message):
        self.message = message
        

class SIDDDataLoader:
    def __init__(self, dname):
        assert dname in ["sidd"], "given dataset name is not valid, supported datasets are ['lol']"  
        #assert type(resize_shape) == int, 'Unknown dtype for resize shape, needed Int' 
        #assert type(batch_size) == int, 'Unknown dtype for batch_size, needed Int' 
        self.dname = dname 
    
    def __image_files(self):
        try:
            with open('SIDD_Small_sRGB_Only/Scene_Instances.txt') as f:
                instances = f.read()

            instances = instances.split('\n')
            path = 'SIDD_Small_sRGB_Only/Data/'

            noisy_images_path = []
            gt_images_path = []

            for f in instances:
                images_path = path + f + '/'

                for g in os.listdir(images_path):
                    image_path = images_path + g

                    if 'NOISY' in image_path:
                        noisy_images_path.append(image_path)
                    else:
                        gt_images_path.append(image_path)

            return noisy_images_path, gt_images_path
        
        except Exception as err:
            return err
            
    def __noisy_image_files(self):
        try:
            num_train = 150
            
            noisy_images_path, _ = self.__image_files()
            train_noisy_data_path = noisy_images_path[: num_train]
            val_noisy_data_path = noisy_images_path[num_train+1: ]

            return train_noisy_data_path, val_noisy_data_path
        
        except Exception as err:
            return err
    
    def __gt_image_files(self):
        try:
            num_train = 150
            
            _, gt_images_path = self.__image_files()
            train_gt_data_path = gt_images_path[: num_train]
            val_gt_data_path = gt_images_path[num_train+1: ]

            return train_gt_data_path, val_gt_data_path
        
        except Exception as err:
            return err
    
    def __train_tf_dataset(self):
        try: 
            noisy_train_files, _ = self.__noisy_image_files()
            gt_train_files, _ = self.__gt_image_files()
            
            tf_dataset = tf.data.Dataset.from_tensor_slices((noisy_train_files, gt_train_files)) 
            return tf_dataset
        
        except Exception as err:
            return err 
    
    def __val_tf_dataset(self):
        try: 
            _, noisy_val_files = self.__noisy_image_files()
            _, gt_val_files = self.__gt_image_files()
            tf_dataset = tf.data.Dataset.from_tensor_slices((noisy_val_files, gt_val_files)) 
            return tf_dataset 
        
        except Exception as err:
            return err
    
    def initialize(self):
        try: 
            if self.dname == "sidd":
                SIDD_DATA_PATH = 'https://competitions.codalab.org/my/datasets/download/a26784fe-cf33-48c2-b61f-94b299dbc0f2'

                if not os.path.exists('SIDD_Small_sRGB_Only/Data'):
                    #os.system(f'wget {SIDD_DATA_PATH}')
                    # https://drive.google.com/file/d/10TC19ND0qeqUCG_mvfhtOjmXkSunYyud/view?usp=sharing
                    url = "https://drive.google.com/uc?id=10TC19ND0qeqUCG_mvfhtOjmXkSunYyud"
                    gdown.download(url)
                    os.system(f'unzip -q smartphone-image-denoising-dataset.zip')
                    os.system(f'rm smartphone-image-denoising-dataset.zip')

                if (os.path.exists("smartphone-image-denoising-dataset.zip") and not os.path.exists("SIDD_Small_sRGB_Only/")):
                    os.system(f'unzip -q smartphone-image-denoising-dataset.zip')
                    os.system(f'rm smartphone-image-denoising-dataset.zip')
                    
        except Exception as err:
            print(err, "rr")
            return err 
        
    def __read_img(self, img_fpath): 
        try: 
            raw = tf.io.read_file(img_fpath)
            image = tf.image.decode_png(raw, channels=3)
            image = tf.image.convert_image_dtype(image, dtype=tf.float32)
            return image
            
        except Exception as err:
            return err
        
    def __load_data(self, lr_img_path, hr_img_path):
        try: 
            lr_img = self.__read_img(lr_img_path)
            hr_img = self.__read_img(hr_img_path)

            # resizing
           # lr_img = tf.image.resize(lr_img, (224, 224))
           # hr_img = tf.image.resize(hr_img, (224, 224))

            return lr_img, hr_img
        
        except Exception as err:
            return err
    
    def image_resize(self, lr_img, hr_img):
        # resizing
        lr_img = tf.image.resize(lr_img, (128, 128))
        hr_img = tf.image.resize(hr_img, (128, 128))
        
        return lr_img, hr_img

    def __create_tf_dataset(self, tf_ds, batch_size, transform):
        
        if transform:
            tf_ds = tf_ds.map(lambda lr, hr: random_crop(lr, hr), num_parallel_calls=tf.data.AUTOTUNE)
            tf_ds = tf_ds.map(random_flip, num_parallel_calls=tf.data.AUTOTUNE)
            tf_ds = tf_ds.map(random_rotate, num_parallel_calls=tf.data.AUTOTUNE)
            tf_ds = tf_ds.batch(batch_size, drop_remainder=False)

        if not transform:
            tf_ds = tf_ds.map(lambda lr, hr: self.image_resize(lr, hr), num_parallel_calls=tf.data.AUTOTUNE)
            tf_ds = tf_ds.batch(1, drop_remainder=False)
        
        tf_ds = tf_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
        
        return tf_ds
        
    def get_dataset(self, subset, batch_size, transform=True):
        assert subset in ("train", 'val'), "unsupported split type"
        try:
            if subset == "train":
                tf_ds = self.__train_tf_dataset()
                tf_ds = tf_ds.map(self.__load_data, num_parallel_calls=tf.data.AUTOTUNE)
                tf_ds = self.__create_tf_dataset(tf_ds, batch_size, transform)
                return tf_ds
            
            else:
                tf_ds = self.__val_tf_dataset()
                tf_ds = tf_ds.map(self.__load_data, num_parallel_calls=tf.data.AUTOTUNE)
                tf_ds = self.__create_tf_dataset(tf_ds, batch_size, transform)
                return tf_ds
                
        except Exception as err:
            print(err)
            raise InitializationErro('DataLoader, has not been initialize, use .initalize method')


In [ ]:
sl = SIDDDataLoader('sidd')

In [ ]:
sl.initialize()

In [ ]:
t_ds = sl.get_dataset("train", 5)

In [ ]:
len(t_ds)

In [ ]:
for i in t_ds:
    print(i[0].shape, i[1].shape)
    #break

In [ ]:
v_ds = sl.get_dataset("val", 5)

In [ ]:
v_ds

In [ ]:
for i in v_ds:
    print(i[0].shape)